# **CNN(Convolutional Neural Network) 구현하기**

**데이터 로드 및 전처리**

라이브러리로부터 MNIST데이터를 로드하고 전처리를 수행한다.

로드한 데이터의 차원은 60000x28x28로 컬러 채널에 대한 차원 수가 나타나있지 않다.

앞서 MLP의 모델을 생성할 때는 입력층(input layer)에 컬러 채널을 명시하지 않았지만 

이번 합성곱 신경망(CNN) 모델에서는 컬러 채널을 1로 명시할 것이다.

 따라서 reshape 함수를 통해 데이터에 1차원의 컬러채널을 추가한다.

MNIST 데이터는 흑백 이미지로 Gray scale을 나타내는 1차원 채널만 추가하면 되지만,

컬러 이미지의 경우 RGB 채널을 나타내는 3차원 채널을 추가해야 한다.

In [1]:
# Import tensorflow packages
import tensorflow as tf
import numpy as np

# MNIST 데이터 로드 및 전처리
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# reshape을 통해 컬러 채널을 나타내는 차원을 추가
print('Before reshaping....')
print(np.shape(x_train))
x_train = x_train.reshape((60000, 28, 28, 1)) # 컬러 이미지의 경우 1이 아닌 3 
x_test = x_test.reshape((10000, 28, 28, 1))
print('After reshaping....')
print(np.shape(x_train))

11501568/11490434 [==============================] - 0s 0us/step
Before reshaping....
(60000, 28, 28)
After reshaping....
(60000, 28, 28, 1)


**합성곱 신경망 모델 생성**

합성곱 신경망은 이미지 처리에 특화된 신경망으로, 합성곱 연산을 통해 이미지의 특징을 추출한다.

add() 메서드를 통해 Sequential 모델에 점진적으로 레이어를 추가하는 방법으로 모델을 구성할 수 있다.

합성곱 신경망에서의 convolutional layer는 Conv2D로 추가할 수 있으며, pooling layer는 MaxPooling2D로 추가할 수 있다.

특징추출을 위한 convolutional layer를 구성한 뒤에는 분류를 위한 fully connected layer(dense layer) 및 출력층을 더한다.

In [2]:
# 빈 모델 생성
model = tf.keras.models.Sequential()

# 합성곱 및 풀링 레이어 추가
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'))

# 완전연결층 및 출력층 추가
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

# 생성된 모델의 구조 확인하기
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

**학습 전략 설정**

최적화 알고리즘 및 손실함수 등을 설정한다.

손실함수인 sparse_categorical_crossentropy는 클래스 레이블이 원핫벡터로 주어지지 않고 정수 형태로 주어질 경우 사용된다.

In [3]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

**모델 학습 및 성능 측정**

앞서 구성한 합성곱 신경망 모델을 학습 데이터로 훈련시키고, 테스트 데이터에 대한 성능을 측정한다.

In [4]:
import time
start_time = time.time()
model.fit(x_train, y_train, batch_size = 100, epochs = 5)
print("모델 학습 완료. 학습 시간 : {}\n".format(time.time() - start_time))

# 테스트 데이터로 학습된 모델의 성능 측정
print('테스트 데이터에 대한 분류 성능')
test_loss, test_acc = model.evaluate(x_test, y_test, verbose = 2)

Epoch 1/5
600/600 [==============================] - 19s 11ms/step - loss: 0.4483 - accuracy: 0.9110
Epoch 2/5
600/600 [==============================] - 7s 11ms/step - loss: 0.0689 - accuracy: 0.9789
Epoch 3/5
600/600 [==============================] - 7s 11ms/step - loss: 0.0491 - accuracy: 0.9843
Epoch 4/5
600/600 [==============================] - 7s 11ms/step - loss: 0.0389 - accuracy: 0.9880
Epoch 5/5
600/600 [==============================] - 7s 11ms/step - loss: 0.0299 - accuracy: 0.9904
모델 학습 완료. 학습 시간 : 45.76169967651367

테스트 데이터에 대한 분류 성능
313/313 - 1s - loss: 0.0564 - accuracy: 0.9834 - 1s/epoch - 4ms/step
